In [ ]:
# Scrape from example website http://quotes.toscrape.com/
# Begin by requesting the home page

from bs4 import BeautifulSoup
import requests

url = 'http://quotes.toscrape.com/'
req = requests.get(url)
len(req.text)

11010

In [ ]:
# Create a session and login to the website using the following credentials:
#   username: [your first name]
#   password: checkpoint

login_url = 'http://quotes.toscrape.com/login'
session_requests = requests.session()
req = session_requests.get(login_url)

soup = BeautifulSoup(req.text, 'lxml')
csrf = soup.findAll("input", {"name": "csrf_token"})[0]['value']

payload = {
    'username': 'Brandon',
    'password': 'checkpoint',
    'csrf_token': csrf
}

req_auth = session_requests.post(
    login_url,
    data = payload,
    headers = dict(referer=login_url)
)

# Print the csrf middleware token assigned during this login

print(csrf)

ykrAUxgpzlTbiDZBGeVIqfNtWvoJLKuOFdmXcYwQPnHjRaMEhSCs


In [ ]:
# Print the headers dictionary from the login request

req.headers

NameError: ignored

In [ ]:
# Scrape the first 10 pages of quotes into a DataFrame including the Quote, Author Name, 
#   Birthdate (on about page), Birth Location, and Tags as columns
 
import pandas as pd

df = pd.DataFrame(columns=['Quote', 'Author Name', 'Birthdate', 'Birth Location', 'Tags'])

url = 'http://quotes.toscrape.com/page/'
about_url = 'https://quotes.toscrape.com'

quotes = []
authors = []
birthdates = []
birthplaces = []
tags = []

for i in range(1, 11):
  u = f'{url}{i}'
  req = requests.get(u)
  html = req.text
  soup = BeautifulSoup(html, 'html.parser')
  for span in soup.findAll('span', {'class':'text'}):
    quotes.append(span.text)
  for small in soup.findAll('small', {'class':'author'}):
    authors.append(small.text)
  for div in soup.findAll('div', {'class':'tags'}):
    temp_tags = []
    for a in div.findAll('a', {'class':'tag'}):
      temp_tags.append(a.text)
    tags.append(temp_tags)

  author_urls = []
  for span in soup.findAll('span'):
    for a in span.findAll('a', {'class':False}, href=True):
      author_urls.append(a['href'])
  for item in author_urls:
    u2 = f'{about_url}{item}'
    req2 = requests.get(u2)
    html2 = req2.text
    soup2 = BeautifulSoup(html2, 'html.parser')
    for span2 in soup2.findAll('span', {'class':'author-born-date'}):
      birthdates.append(span2.text)
    for span2 in soup2.findAll('span', {'class':'author-born-location'}):
      birthplaces.append(span2.text[3:])

df["Quote"] = quotes
df["Author Name"] = authors
df["Birthdate"] = birthdates
df["Birth Location"] = birthplaces
df["Tags"] = tags

# Sort the DataFrame by Birth Location descending
# Print the first 20 rows of the resulting DataFrame

df.sort_values(by=['Birth Location'], ascending=False, inplace=True)
df.head(20)

,Quote,Author Name,Birthdate,Birth Location,Tags
65,"“Do not pity the dead, Harry. Pity the living,...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[live-death-love]
31,"“Of course it is happening inside your head, H...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[dumbledore]
90,"“The truth."" Dumbledore sighed. ""It is a beaut...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[truth]
27,“It is impossible to live without failing at s...,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[]
48,"“It matters not what someone is born, but what...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[dumbledore]
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...","[abilities, choices]"
11,“It takes a great deal of bravery to stand up ...,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...","[courage, friends]"
82,"“Remember, if the time should come when you ha...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[integrity]
34,"“To the well-organized mind, death is but the ...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...","[death, inspirational]"
9,"“A day without sunshine is like, you know, nig...",Steve Martin,"August 14, 1945","Waco, Texas, The United States","[humor, obvious, simile]"


In [ ]:
# Print the number of times each Author shows up in the dataframe

df["Author Name"].value_counts()

Albert Einstein           10
J.K. Rowling               9
Marilyn Monroe             7
Mark Twain                 6
Dr. Seuss                  6
C.S. Lewis                 5
Jane Austen                5
Bob Marley                 3
Ralph Waldo Emerson        2
Mother Teresa              2
Eleanor Roosevelt          2
Charles Bukowski           2
Suzanne Collins            2
Ernest Hemingway           2
George R.R. Martin         2
Pablo Neruda               1
Alexandre Dumas fils       1
J.M. Barrie                1
Alfred Tennyson            1
Madeleine L'Engle          1
Charles M. Schulz          1
Terry Pratchett            1
Haruki Murakami            1
Steve Martin               1
Douglas Adams              1
William Nicholson          1
John Lennon                1
Martin Luther King Jr.     1
Jimi Hendrix               1
Allen Saunders             1
J.D. Salinger              1
Ayn Rand                   1
George Eliot               1
George Bernard Shaw        1
E.E. Cummings 